INSTALL ALBUMENTATIONS (TIVE QUE USAR ESSES ARGUMENTOS POR JÁ TER OPENCV INSTALADO NO SISTEMA)

In [2]:
!pip install -q -U albumentations --no-binary qudida,albumentations


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


In [4]:
import cv2
import albumentations as A
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import csv
import random
#from google.colab.patches import cv2_imshow

OPEN CSV COM ANOTAÇÕES DE BOUNDING BOXES

In [ ]:
images = []
trainRows = []
allRows = []

with open('/content/drive/MyDrive/CSVs/Modelo5.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        
        allRows.append(row)
        checkTrain = row[0]

        if checkTrain == 'TRAIN':
            trainRows.append(row)

        if not images:
            if checkTrain == 'TRAIN':
                images.append(row[1])

        elif row[1] != images[-1] and checkTrain == 'TRAIN':
            images.append(row[1])


FAZ AUGMENTATIONS DE FLIP, BLUR E BRIGHTNESS E SALVA FOTOS NA PASTA AUGMENTATION

In [ ]:

pastaDrive = '/content/drive/MyDrive/images_16.02.22/augmentation'



for image in images:
    split = image.split("/")
    nomePasta = split[5]
    nomeFoto = split[6]

    bboxes = []


    for row in trainRows:
    if image == row[1]:
        temp = []
        temp.append(float(row[3]))
        temp.append(float(row[4]))
        temp.append(float(row[7]))
        temp.append(float(row[8]))
        bboxes.append(temp)

    img = cv2.imread(image)


    transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.OneOf([
        A.Blur(p=0.3),
        A.MedianBlur(p=0.3),
        A.MotionBlur(p=0.3),
    ], p=0.8),
    A.OneOf([
        A.RandomBrightnessContrast(p=1.0),
    ], p=0.8),

    # A.ISONoise(p=1.0),
    # A.RandomRain(p=1.0, drop_length=27, blur_value=5, brightness_coefficient=0.7)
    # A.RandomSunFlare(p=1.0)
    # A.Perspective(p=1.0)
    ],bbox_params=A.BboxParams(format='albumentations', label_fields=[]))


    images_list = []
    bboxes_list = []

    for i in range(3):
        transformed = transform(image=img, bboxes=bboxes)
        transformed_img = transformed["image"]
        transformed_bboxes = transformed["bboxes"]

        images_list.append(transformed_img)
        bboxes_list.append(transformed_bboxes)


    photoCount = 1
    j = 0
    for imageT in images_list:
        nomeFotoT = pastaDrive + '/' + nomePasta + '/' + nomeFoto[:-4] + '_T' + str(photoCount) + '.jpg'
        cv2.imwrite(nomeFotoT, imageT)
        height = imageT.shape[0]
        width = imageT.shape[1]
        for bboxes in bboxes_list[j]:  
            x_min = bboxes[0]
            y_min = bboxes[1]
            x_max = bboxes[2]
            y_max = bboxes[3]

            ymin = int(max(1,(y_min * height)))
            xmin = int(max(1,(x_min * width)))
            ymax = int(min(height,(y_max * height)))
            xmax = int(min(width,(x_max * width)))

            data = ["TRAIN", nomeFotoT, 'log', x_min, y_min, x_max, y_min, x_max, y_max, x_min, y_max]

            allRows.append(data)

            cv2.rectangle(imageT, (xmin,ymin), (xmax,ymax), (10, 255, 0), 2)


    photoCount += 1
    j += 1
    # cv2_imshow(imageT)


SHUFFLE NO DATASET

In [ ]:
newUniqueRows = []
newRowsCount = []
newRows = []


for x in allRows:

    newRowsCount.append(x)

    linha = x[1]

    if not newUniqueRows:
        newUniqueRows.append(linha)

    elif newUniqueRows[-1] != linha:
        newUniqueRows.append(linha)


#shuffle
random.shuffle(newUniqueRows)


for index, i in enumerate(newUniqueRows):
    for j in newRowsCount:
        if i == j[1]:
            data = [j[0], j[1], j[2], j[3], j[4], j[5], j[6], j[7], j[8], j[9], j[10]]
            newRows.append(data)

SALVA CSV COM TODAS AS BOUNDING BOXES

In [ ]:
with open('/content/drive/MyDrive/CSVs/Modelo5_T.csv', 'w', newline='') as f:

    for data in newRows:

        writer = csv.writer(f)

        writer.writerow(data)